# **CIS 520: Machine Learning, Fall 2020**
# **Week 3, Worksheet 3**
## **Bias and Variance**


- **Content Creators:** Kenneth Shinn
- **Content Reviewers:** Lyle Ungar, Michael Zhou
    
As we know from lecture, there is a tradeoff between a model's bias and variance. In this worksheet, we take a look at this trade off in action. 

<h5> Objectives: </h5> 
<ul>
    <li>See how model complexity affects bias and variance. </li>
    <li>See how the number of observations affects variance</li>
</ul>
       

In [ ]:
# run below
!pip install mlxtend
%pip install mlxtend --upgrade

# if you're on a mac, and the above throws an error, uncomment and run below
# pip3 install mlxtend

In [ ]:
import pandas as pd
import numpy as np
from random import uniform
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Ridge
from sklearn.preprocessing import PolynomialFeatures
from sklearn.metrics import mean_squared_error
import statistics 
from mlxtend.evaluate import bias_variance_decomp

First let's construct a model and generate some data. The function below uses the simple model $y = x$, and adds an error term $\epsilon \sim N(0,25)$. Our goal over this worksheet is to see how adjusting model complexity and $n$ (the number of observations) changes model bias and variance. 

In [ ]:
# let's generate some data
# returns X_train, y_train, X_test, y_test 
def generate_data(n_train, n_test):
    X_train = []
    y_train = []
    X_test = []
    y_test = []
    
    all_x_train = np.linspace(-10,10,n_train)
    all_x_test = np.linspace(-10, 10, n_test)
    
    for x in all_x_train:
        X_train.append(x)
        y_train.append(x + np.random.normal(0, 5^2))

    for x in all_x_test:
        X_test.append(x)
        y_test.append(x + np.random.normal(0, 5^2))
        
    X_train = np.array(X_train).reshape(-1, 1)
    y_train = np.array(y_train).reshape(-1, 1)
    X_test = np.array(X_test).reshape(-1, 1)
    y_test = np.array(y_test).reshape(-1, 1)
    
    return X_train, y_train, X_test, y_test


# Bias vs. Variance (Model Complexity)

Here the model complexity will be determined by the polynominal degree. That is, a polynominal of degree 2 will assume the model to be $y = \beta_0 + \beta_1 x + \beta_2 x^2$.

The code below will pull some data from the above data generator and train a model of degree $d$. You can run the code multiple times to see how the model will change depending on the degree of the trained model. Now consider the following questions:

* Do you expect a higher or lower degree to result in a greater change between instances?
* From our theory, would you expect the bias to go up or down as the degree increases? How about the variance? What about the MSE?

In [ ]:
# change the parameters here, d is the degree, n is the number of observations
d = 2
n = 10
# *****************************

X_train, y_train, X_test, y_test = generate_data(n, n)
X_train_poly = PolynomialFeatures(degree=d).fit_transform(X_train)
X_test_poly = PolynomialFeatures(degree=d).fit_transform(X_test)

lm = LinearRegression()
lm.fit(X_train_poly,y_train)
y_pred_poly = lm.predict(X_train_poly)

sorted_zip = sorted(zip(X_train,y_pred_poly), key = lambda x: x[0])
X_train, y_pred_poly = zip(*sorted_zip)

plt.figure(figsize=(20, 10))
plt.title('Degree ' + str(d) + ' model with training observations')
plt.scatter(X_train, y_train)
plt.plot(X_train, y_pred_poly, color = 'red')
plt.show()
mse, bias, var = bias_variance_decomp(lm, X_train_poly, y_train.flatten(), X_test_poly, y_test.flatten(), loss='mse', num_rounds=200, random_seed=1)

print('MSE: ' + str("%.2f" % mse))
print('Bias: ' + str("%.2f" %bias))
print('Variance: ' + str("%.2f" %var))

## Plotting MSE, Bias, and Variance

The below code and graph generate 15 models with varying degrees from 0 to 6 and finds the average bias, variance, and MSE for each model complexity (defined by degree). The purpose of this is to "test" the claims that:

<ol>
    <li>Bias goes down</li>
    <li>Variance goes up</li>
    <li>MSE goes down, then up</li>
</ol>

as the model complexity increases. 

In [ ]:
degrees = [0, 1, 2, 3, 4, 5, 6]
num_of_iterations = 15
n = 50

In [ ]:
avg_mse_list = []
avg_bias_list = []
avg_variance_list = []

for d in degrees:
    mse_list = []
    bias_list = []
    variance_list = []
    for i in range(num_of_iterations):
        X_train, y_train, _, _ = generate_data(n, n)
        X_train_poly = PolynomialFeatures(degree=d).fit_transform(X_train)
        X_test_poly = PolynomialFeatures(degree=d).fit_transform(X_test)

        lm = LinearRegression()
        mse, bias, var = bias_variance_decomp(lm, X_train_poly, y_train.flatten(), X_test_poly, y_test.flatten(), loss='mse', num_rounds=1000, random_seed=1)

        mse_list.append(mse)
        bias_list.append(bias)
        variance_list.append(var)
    
    avg_mse_list.append(statistics.mean(mse_list))
    avg_bias_list.append(statistics.mean(bias_list))
    avg_variance_list.append(statistics.mean(variance_list)) 

plt.figure(figsize=(20, 10))
plt.plot(avg_mse_list, color = 'red',label="MSE")
plt.title('Graph of MSE Across Different Model Complexities')
plt.legend()
plt.show()

plt.figure(figsize=(20, 10))
plt.plot(avg_bias_list, color = 'green', label= "Bias")
plt.title('Graph of Bias Across Different Model Complexities')
plt.legend()
plt.show()


plt.figure(figsize=(20, 10))
plt.plot(avg_variance_list, color = 'blue', label= "Variance")
plt.title('Graph of Variance Across Different Model Complexities')
plt.legend()
plt.show()

# Regularization and Bias/Variance

Now, let's take a look at how regularization affects model bias and variance. Here, we will use Ridge regression (L2 regularization) and adjust the regularization strength $\lambda$ to see how it affects bias and variance. 

Below, adjust $\lambda$ and see how it affects the slope of the linear model, as well as the bias and variance. 

* Before running any code, as we increase $\lambda$, what changes do you expect to see in the slope, bias, and variance?

In [ ]:
# change the parameters here, lmda is lambda, n is the number of observations
lmda = 1
n = 100
# *****************************

X_train, y_train, X_test, y_test = generate_data(n, n)

rlm = Ridge(lmda)
rlm.fit(X_train,y_train)
y_pred = rlm.predict(X_train)

plt.figure(figsize=(20, 10))
plt.title('Ridge Regression Model with Lambda = ' + str(lmda) + ' (includes training observations)')
plt.scatter(X_train, y_train)
plt.plot(X_train, y_pred, color = 'red')
plt.show()

mse, bias, var = bias_variance_decomp(rlm, X_train, y_train.flatten(), X_test, y_test.flatten(), loss='mse', num_rounds=200, random_seed=1)

print('MSE: ' + str("%.2f" %mse))
print('Bias: ' + str("%.2f" %bias))
print('Variance: ' + str("%.2f" %var))

# Model Variance vs. Number of Observations

Now, let's see how the variance of a model can decrease as the number of observations increases. For this example, we will can pin down the model and adjust the number of observations. From our theory, we know that the model variance will go down as we increase the number of observations. Let's see this in action below. 

In [ ]:
# change the parameters here, d is the degree, n is the number of observations
d = 2
n = 5
# *****************************

X_train, y_train, X_test, y_test = generate_data(n, n)
X_train_poly = PolynomialFeatures(degree=d).fit_transform(X_train)
X_test_poly = PolynomialFeatures(degree=d).fit_transform(X_test)

lm = LinearRegression()
lm.fit(X_train_poly,y_train)
y_pred_poly = lm.predict(X_train_poly)

sorted_zip = sorted(zip(X_train,y_pred_poly), key = lambda x: x[0])
X_train, y_pred_poly = zip(*sorted_zip)

plt.figure(figsize=(20, 10))
plt.title('Degree ' + str(d) + ' model with ' + str(n) + ' training observations')
plt.scatter(X_train, y_train)
plt.plot(X_train, y_pred_poly, color = 'red')
plt.show()
mse, bias, var = bias_variance_decomp(lm, X_train_poly, y_train.flatten(), X_test_poly, y_test.flatten(), loss='mse', num_rounds=200, random_seed=1)

print('MSE: ' + str("%.2f" %mse))
print('Bias: ' + str("%.2f" %bias))
print('Variance: ' + str("%.2f" %var))

## Visualizing the effects of $n$ on variance

Below, we can visualize this decrease in variance by seeing how different instances of data can generate more similar models as the number of observations increase. Here, we keep the model degree constant (5), and vary the number of observations. Given the model complexity and the number of observations, data is drawn 5 times and the resulting trained models are graphed. 

In [ ]:
degrees = 5
num_of_iterations = 5
num_of_observations = [10, 100, 1000, 10000, 100000]

In [ ]:
for n in num_of_observations:
    plt.figure(figsize=(20, 10))
    for _ in range(num_of_iterations):
        X_train, y_train, X_test, y_test = generate_data(n, n)
        X_train_poly = PolynomialFeatures(degree=degrees).fit_transform(X_train)

        lm = LinearRegression()
        lm.fit(X_train_poly,y_train)
        y_pred_poly = lm.predict(X_train_poly)
        sorted_zip = sorted(zip(X_train,y_pred_poly), key = lambda x: x[0])
        X_train, y_pred_poly = zip(*sorted_zip)
        plt.plot(X_train, y_pred_poly)
    
    plt.title(str(num_of_iterations) + ' Polynomial Regression Models of Degree ' + str(degrees) + ', Different Data Observation Instances From Same Data Generating Model: n = '+ str(n) )
    plt.show()


# Exercises

* What do you see to be the relationship between the number of observations and variance of the models? What might happen if we decrease the model complexity? Do you think that it will "converge" faster or slower? Go ahead and try it out.
* What does this worksheet tell you about what needs to be considered when creating and using machine learning models? 